In [1]:
from torch_geometric.datasets import WebKB
from sklearn.metrics import accuracy_score
import random
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Data preprocessing

In [2]:
dataset = WebKB(root='./', name='Cornell')
data = dataset[0]

Transpose original label to binary label, and do train-test split

In [3]:
data.y[data.y > 0] = 1
data_gt = data.y.detach().cpu().numpy()

n_node = len(data.x)
n_test = n_node // 10

all_idx = list(range(n_node))
random.shuffle(all_idx)
test_index = all_idx[:n_test]
train_index = all_idx[n_test:]
train_data = data.subgraph(torch.LongTensor(train_index))

print(train_data)

Data(x=[165, 1703], edge_index=[2, 257], y=[165], train_mask=[165, 10], val_mask=[165, 10], test_mask=[165, 10])


## Classification algorithm

Define the network to do classification.

In [4]:
class NodeClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.fc = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        x = self.fc(x)
        return x

class RelationClassifier(nn.Module):
    def __init__(self, nodecls, zinput_dim, output_dim):
        super().__init__()
        self.nodecls = nodecls
        self.fcz = nn.Linear(zinput_dim, output_dim)
    
    def forward(self, x, z):
        x = self.nodecls(x)
        z = self.fcz(z)
        return (x + z)

### Phase 1

Step 1. Train $\phi_1(f_v)$ to predict $Y_v$ based on $f_v$

In [5]:
input_dim = data.x.shape[1]
output_dim = 2

model_f = NodeClassifier(input_dim, output_dim)
optim_f = optim.Adam(model_f.parameters())
loss_fn = nn.CrossEntropyLoss()

In [6]:
epochs = 5
for epoch in tqdm(range(epochs)):
    model_f.train()
    optim_f.zero_grad()

    logits = model_f(train_data.x)
    loss = loss_fn(logits, train_data.y)

    loss.backward()
    optim_f.step()

100%|██████████| 5/5 [00:00<00:00, 833.39it/s]


Step 2. Train $\phi(f_v,z_v)$ to predict $Y_v$ based on $f_v$ and summary $z_v$

In [7]:
model_z = RelationClassifier(model_f, 4, output_dim)
optim_z = optim.Adam(model_z.parameters())

Build neighbouring feature $z_v$

In [8]:
train_data.feature_z = torch.zeros(train_data.num_nodes, 4)
for v in range(train_data.num_nodes):
    in_neighs = train_data.edge_index[1, train_data.edge_index[1] == v]
    out_neighs = train_data.edge_index[1, train_data.edge_index[0] == v]
    in_labels = set(train_data.y[in_neighs].detach().numpy())
    out_labels = set(train_data.y[out_neighs].detach().numpy())
    
    train_data.feature_z[v] = torch.tensor([
        0 in in_labels,
        1 in in_labels,
        0 in out_labels,
        1 in out_labels
    ])

In [9]:
epochs = 5

for epoch in tqdm(range(epochs)):
    model_z.train()
    optim_z.zero_grad()

    logits = model_z(train_data.x, train_data.feature_z)
    loss = loss_fn(logits, train_data.y)

    loss.backward()
    optim_z.step()
    

100%|██████████| 5/5 [00:00<00:00, 1000.12it/s]


## Phase 2

Step 1. Set initial $Y_v$ by $\phi_1$

In [10]:
model_f.eval()
with torch.no_grad():
    data.y = model_f(data.x).argmax(dim=1)
accuracy = accuracy_score(data_gt, data.y)
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.8032786885245902


Step 2. Repeat the iteration:
- Update $z_v$ by $Y_{N(v)}$
- Update $Y_v$ with new $z_v$

In [11]:
epochs = 5
for epoch in tqdm(range(epochs)): 
    data.feature_z = torch.zeros(data.num_nodes, 4)
    for v in range(data.num_nodes):
        in_neighs = data.edge_index[1, data.edge_index[1] == v]
        out_neighs = data.edge_index[1, data.edge_index[0] == v]
        in_labels = set(data.y[in_neighs].detach().numpy())
        out_labels = set(data.y[out_neighs].detach().numpy())
        
        data.feature_z[v] = torch.tensor([
            0 in in_labels,
            1 in in_labels,
            0 in out_labels,
            1 in out_labels
        ])
    
    model_z.eval()
    with torch.no_grad():
        data.y = model_z(data.x, data.feature_z).argmax(dim=1)

100%|██████████| 5/5 [00:00<00:00, 47.10it/s]


In [12]:
accuracy = accuracy_score(data_gt, data.y)
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.8633879781420765


Conclusion: In this case, iterative classification performs better than classification with only features, however
- The models are not well trained, because in this dataset, when train epoch is set larger, even classification with only features can reach acc 100%
- Sometimes iterative classification performs worse, because dimension of additional features(4) is greatly less than that of initial feature(1703)